In [1]:
from db_queries import get_population, get_ids
from db_queries import get_location_metadata as get_locs

In [2]:
import pandas as pd, numpy as np

# LSFF: choose population coverage data by hand for tier 2 countries

## vehicles: Wheat flour, maize flour, oil

## countries: Bangladesh, Pakistan, United Republic of Tanzania, Uganda, South Africa

In [3]:
nutrients = ['iron','zinc','folic acid','vitamin a']

In [4]:
data_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_03_16_2021.csv'
assm_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/extraction_sheet_lsff_assumed_coverage_03_11_2021.csv'

df = pd.read_csv(data_path)

In [5]:
df.loc[df.nutrient=="folic acid, folate, b9",'nutrient']= 'folic acid'

In [6]:
assum = pd.read_csv(assm_path)

In [7]:
# don't need to deal with this for tier2
assum.location_name.unique()

array(['Ethiopia', 'Myanmar', 'India'], dtype=object)

In [8]:
df['estimation_status'] = 'na'

In [9]:
df['data_choice_notes'] = ""

In [10]:
mult_estimates_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/lsff_data_estimated_03_11_2021.csv'

mult_estimates = pd.read_csv(mult_estimates_path)

In [11]:
mult_estimates[mult_estimates.location_name=="Pakistan"]

,location_name,vehicle,nutrient,standard,A,B,B_estimate,C,C_estimate
255,Pakistan,wheat flour,iron,Mandatory in Punjab only,97.333333,48.666667,0.0,5.333333,0.0
256,Pakistan,wheat flour,folic acid,Mandatory in Punjab only,97.333333,48.666667,0.0,1.946667,1.0
257,Pakistan,wheat flour,zinc,Unknown,97.333333,48.666667,0.0,1.946667,1.0
258,Pakistan,wheat flour,vitamin b12,Unknown,97.333333,48.666667,0.0,1.946667,1.0
259,Pakistan,wheat flour,vitamin b1,Unknown,97.333333,48.666667,0.0,0.000000,1.0
260,Pakistan,oil,vitamin a,Mandatory,100.000000,95.500000,0.0,30.000000,0.0
261,Pakistan,oil,vitamin d,Mandatory,100.000000,95.500000,0.0,NaN,NaN
348,Pakistan,wheat flour,vitamin a,NaN,97.333333,48.666667,0.0,0.000000,1.0


In [12]:
#reformat
mult_estimates.loc[(mult_estimates.B_estimate!=1.0),'B'] = np.nan
mult_estimates.loc[(mult_estimates.C_estimate!=1.0),'C'] = np.nan

mult_estimates = pd.melt(mult_estimates,
                         id_vars = ['location_name','vehicle','nutrient','standard'],
                         value_vars = ['B','C'], var_name = 'value_description', value_name = 'value_mean').dropna()

mult_estimates.loc[(mult_estimates.value_description=="B"),'nutrient'] = 'NA'

mult_estimates = mult_estimates.drop_duplicates()

mult_estimates.value_description = mult_estimates.value_description.map({
    'B':'percent of population eating industrially produced vehicle',
    'C':'percent of population eating fortified vehicle'
})

In [13]:
mult_estimates['estimation_status'] = 'multiplicative'

In [14]:
## load legal combos
import pickle
data_prep_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/inputs/'

with open(data_prep_dir + 'lsff_vehicle_nutrient_pairs.pickle', 'rb') as handle:
    vehicle_nutrient_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_country_vehicle_pairs.pickle', 'rb') as handle:
    country_vehicle_map = pickle.load(handle)
    
with open(data_prep_dir + 'lsff_vehicle_country_pairs.pickle', 'rb') as handle:
    vehicle_country_map = pickle.load(handle)

In [15]:
location_names = ['Bangladesh','Pakistan','United Republic of Tanzania','Uganda','South Africa']

In [16]:
# these are the vehicles per country we need
target_high_level = pd.DataFrame([(loc,v) for loc in location_names for v in country_vehicle_map[loc]],
            columns=['location_name','vehicle']).sort_values(['location_name','vehicle']).set_index(['location_name','vehicle'])

target_high_level

Empty DataFrame
Columns: []
Index: [(Bangladesh, oil), (Bangladesh, wheat flour), (Pakistan, oil), (Pakistan, wheat flour), (South Africa, maize flour), (South Africa, oil), (South Africa, wheat flour), (Uganda, maize flour), (Uganda, oil), (Uganda, wheat flour), (United Republic of Tanzania, maize flour), (United Republic of Tanzania, oil), (United Republic of Tanzania, wheat flour)]

In [17]:
target_vehicles = [i for i in vehicle_nutrient_map.keys() if 'iron' in vehicle_nutrient_map[i]]

target_a = pd.DataFrame([(loc,vehicle,nutrient,'percent of population eating fortified vehicle') for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for nutrient in vehicle_nutrient_map[vehicle]],
            columns=['location_name','vehicle','nutrient','value_description']).sort_values(['location_name','vehicle','nutrient'])
target_a = target_a[(target_a.nutrient.isin(nutrients))]

target_b = pd.DataFrame([(loc,vehicle,'na',val) for loc in location_names
                       for vehicle in country_vehicle_map[loc]
                      for val in ['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle']],
            columns=['location_name','vehicle','nutrient','value_description'])

sortvars = ['location_name','vehicle','value_description','nutrient']
target = target_a.append(target_b).sort_values(sortvars).set_index(sortvars)

In [18]:
target = target.reset_index()
rcols = target.columns.tolist()

In [19]:
check_cols = ['location_id','location_name','urbanicity','subnational_name','vehicle','value_description','nutrient','value_mean','value_025_percentile',
       'value_975_percentile','sub_population','source_year','notes','source_citation','source_link','data_choice_notes']

def filter_data(country, vehicle, val):    
    output = df.loc[(df.location_name==country)
           & (df.vehicle==vehicle)
           & (df.value_description==val)
           & (df.value_mean.notna()),check_cols]
    
    return output

In [20]:
def check_one_country(country):
    vehicles = ['oil', 'wheat flour', 'salt', 'maize flour', 'rice', 'bouillon']
    values_gold = ['percent of population eating fortified vehicle',
               'percent of population eating industrially produced vehicle',
               'percent of population eating vehicle']
    return pd.concat([filter_data(country, vehicle, val) for vehicle in vehicles for val in values_gold])

In [21]:
usecols = ['location_id','location_name','subnational_name','vehicle','value_description','nutrient','value_mean', 'value_025_percentile',
       'value_975_percentile']
subset_data = {}

In [22]:
for i in location_names:
    subset_data[i] = pd.DataFrame()

## Bangladesh

In [23]:
bangladesh = check_one_country('Bangladesh')

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v107/lib/python3.7/site-packages/pandas/core/indexing.py:1418: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [24]:
bangladesh.value_description.unique()

array(['percent of population eating industrially produced vehicle',
       'percent of population eating vehicle'], dtype=object)

In [25]:
mult_estimates[(mult_estimates.location_name=="Bangladesh")]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
12,Bangladesh,wheat flour,NA,Voluntary,percent of population eating industrially prod...,38.241333,multiplicative
363,Bangladesh,wheat flour,iron,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
364,Bangladesh,wheat flour,zinc,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
365,Bangladesh,wheat flour,folic acid,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
366,Bangladesh,wheat flour,vitamin b12,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
367,Bangladesh,wheat flour,vitamin b1,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
368,Bangladesh,wheat flour,vitamin a,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
369,Bangladesh,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,42.849572,multiplicative
370,Bangladesh,oil,vitamin d,Unknown,percent of population eating fortified vehicle,0.000000,multiplicative


In [26]:
## pct of pop eating industrially produced oil

In [27]:
# pct of pop eating industrially produced oil
bangladesh[(bangladesh.vehicle=="oil") & (bangladesh.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
35,NaN,Bangladesh,mixed/both,na,oil,percent of population eating industrially prod...,na,88.4,84.5,92.3,total population,2015,"survey pop = ""households and women of reprod a...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
199,NaN,Bangladesh,NaN,na,oil,percent of population eating industrially prod...,na,87.5,na,na,total population,2015,no uncertainty,"Raghavan R, Aaron GJ, Nahar B, et al. Househol...",https://journals.plos.org/plosone/article?id=1...,


In [28]:
bangladesh.loc[(bangladesh.vehicle=="oil") &
           (bangladesh.value_description=="percent of population eating industrially produced vehicle") &
          (bangladesh.urbanicity=="mixed/both"),'data_choice_notes'] = "Keeping Aaron number (88%) and descarding Raghavan number (87%), as they are very similar, from the same year, and the Aaron number has uncertainty"

In [29]:
# keep the row with uncertainty, as values are very similar and from the same year

subset_data['Bangladesh'] = subset_data['Bangladesh'].append(bangladesh[(bangladesh.vehicle=="oil") &
           (bangladesh.value_description=="percent of population eating industrially produced vehicle") &
          (bangladesh.urbanicity=="mixed/both")])

In [30]:
bangladesh[(bangladesh.vehicle=="oil") & (bangladesh.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
19,NaN,Bangladesh,unknown,Barisal,oil,percent of population eating vehicle,na,95,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
20,NaN,Bangladesh,unknown,Chittagong,oil,percent of population eating vehicle,na,89,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
21,NaN,Bangladesh,unknown,Dhaka,oil,percent of population eating vehicle,na,75,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
22,NaN,Bangladesh,unknown,Khulna,oil,percent of population eating vehicle,na,95,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
23,NaN,Bangladesh,mixed/both,Rajshahi,oil,percent of population eating vehicle,na,51,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
24,NaN,Bangladesh,mixed/both,Rangpur,oil,percent of population eating vehicle,na,59,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
25,NaN,Bangladesh,mixed/both,Sylhet,oil,percent of population eating vehicle,na,68,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
33,NaN,Bangladesh,unknown,na,oil,percent of population eating vehicle,na,76,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
34,NaN,Bangladesh,mixed/both,na,oil,percent of population eating vehicle,na,100,na,na,total population,2015,"no uncertainty. survey pop = ""households and w...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
38,NaN,Bangladesh,mixed/both,na,oil,percent of population eating vehicle,na,99.8,na,na,total population,2011,NaN,"Leyvraz M, Laillou A, Rahman S, et al. An Asse...",https://www.researchgate.net/publication/28558...,


In [31]:
# percent of pop eating oil
bangladesh.loc[(bangladesh.vehicle=="oil") & (bangladesh.value_description=="percent of population eating vehicle") &
          (bangladesh.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245753"),'data_choice_notes'] = "Keeping Aaron (survey data) number from 2015 (100%), discarding Fiedler number (51-90%) from 2005, and discarding Levyraz number from 2011 (99.8%)"

In [32]:
subset_data['Bangladesh'] = subset_data['Bangladesh'].append(bangladesh[(bangladesh.vehicle=="oil") & (bangladesh.value_description=="percent of population eating vehicle") &
          (bangladesh.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245753")])

In [33]:
# percent of pop eating wheat flour
bangladesh[(bangladesh.vehicle=="wheat flour") & (bangladesh.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
26,NaN,Bangladesh,mixed/both,Barisal,wheat flour,percent of population eating vehicle,na,40,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
27,NaN,Bangladesh,mixed/both,Dhaka,wheat flour,percent of population eating vehicle,na,63,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
28,NaN,Bangladesh,mixed/both,Khulna,wheat flour,percent of population eating vehicle,na,59,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
29,NaN,Bangladesh,unknown,Rajshahi,wheat flour,percent of population eating vehicle,na,70,na,na,NaN,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
30,NaN,Bangladesh,unknown,Rangpur,wheat flour,percent of population eating vehicle,na,70,na,na,NaN,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
31,NaN,Bangladesh,unknown,Sylhet,wheat flour,percent of population eating vehicle,na,77,na,na,NaN,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
32,NaN,Bangladesh,mixed/both,na,wheat flour,percent of population eating vehicle,na,65,na,na,total population,2005,no uncertainty,"Fiedler JL, Lividini K, Guyondet C, Bermudez O...",https://journals.sagepub.com/doi/pdf/10.1177/1...,
36,NaN,Bangladesh,mixed/both,na,wheat flour,percent of population eating vehicle,na,76.2,na,na,under-5,2011,no uncertainty,NaN,https://www.researchgate.net/publication/28558...,
37,NaN,Bangladesh,mixed/both,na,wheat flour,percent of population eating vehicle,na,41.9,na,na,women of reproductive age,2011,no uncertainty,NaN,https://www.researchgate.net/publication/28558...,
203,NaN,Bangladesh,NaN,na,wheat flour,percent of population eating vehicle,NaN,41.9,na,na,women of reproductive age,2011,no uncertainty,"Leyvraz M, Laillou A, Rahman S, et al. An Asse...",https://www.mdpi.com/2072-6643/8/9/541/htm,


In [34]:
## keep the more recent, u5- and wra- specific vars}
bangladesh.loc[(bangladesh.vehicle=="wheat flour") & 
           (bangladesh.value_description=="percent of population eating vehicle") &
          (bangladesh.source_citation=="Leyvraz M, Laillou A, Rahman S, et al. An Assessment of the Potential Impact of Fortification of Staples and Condiments on Micronutrient Intake of Young Children and Women of Reproductive Age in Bangladesh. Nutrients 2016; 8: 541."),"data_choice_notes"] = "Keeping Levyraz numbers (42% and 76%), as they are u5- and WRA- specfic and more recent (2011). Discarding 2005 Fiedler all-populaion numbers"

In [35]:
subset_data['Bangladesh'] = subset_data['Bangladesh'].append(bangladesh[(bangladesh.vehicle=="wheat flour") & 
           (bangladesh.value_description=="percent of population eating vehicle") &
          (bangladesh.source_citation=="Leyvraz M, Laillou A, Rahman S, et al. An Assessment of the Potential Impact of Fortification of Staples and Condiments on Micronutrient Intake of Young Children and Women of Reproductive Age in Bangladesh. Nutrients 2016; 8: 541.")])

In [36]:
# ## need to use estimates for
# - pct eating fortified oil
# - pct eating fortifiable wheat
# - pct eating fortified wheat

In [37]:
subset_data['Bangladesh'] = subset_data['Bangladesh'].append(mult_estimates[(mult_estimates.location_name=="Bangladesh") 
               & (mult_estimates.vehicle=="oil")
              & (mult_estimates.nutrient=="vitamin a")])

mult_estimates[(mult_estimates.location_name=="Bangladesh") 
               & (mult_estimates.vehicle=="oil")
              & (mult_estimates.nutrient=="vitamin a")]

/ihme/code/central_comp/miniconda_svc-ccomp/envs/v107/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
369,Bangladesh,oil,vitamin a,Mandatory,percent of population eating fortified vehicle,42.849572,multiplicative


In [38]:
subset_data['Bangladesh'] = subset_data['Bangladesh'].append(mult_estimates[(mult_estimates.location_name=="Bangladesh") 
               & (mult_estimates.vehicle=="wheat flour")
              & (mult_estimates.nutrient.isin(['NA','iron','zinc','folic acid','vitamin a']))])

mult_estimates[(mult_estimates.location_name=="Bangladesh") 
               & (mult_estimates.vehicle=="wheat flour")
              & (mult_estimates.nutrient.isin(['NA','iron','zinc','folic acid','vitamin a']))]

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
12,Bangladesh,wheat flour,NA,Voluntary,percent of population eating industrially prod...,38.241333,multiplicative
363,Bangladesh,wheat flour,iron,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
364,Bangladesh,wheat flour,zinc,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
365,Bangladesh,wheat flour,folic acid,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative
368,Bangladesh,wheat flour,vitamin a,Voluntary,percent of population eating fortified vehicle,0.155875,multiplicative


In [39]:
subset_data['Bangladesh'][rcols + ['value_mean']].sort_values(rcols)

,location_name,vehicle,value_description,nutrient,value_mean
369,Bangladesh,oil,percent of population eating fortified vehicle,vitamin a,42.8496
35,Bangladesh,oil,percent of population eating industrially prod...,na,88.4
34,Bangladesh,oil,percent of population eating vehicle,na,100
365,Bangladesh,wheat flour,percent of population eating fortified vehicle,folic acid,0.155875
363,Bangladesh,wheat flour,percent of population eating fortified vehicle,iron,0.155875
368,Bangladesh,wheat flour,percent of population eating fortified vehicle,vitamin a,0.155875
364,Bangladesh,wheat flour,percent of population eating fortified vehicle,zinc,0.155875
12,Bangladesh,wheat flour,percent of population eating industrially prod...,NA,38.2413
203,Bangladesh,wheat flour,percent of population eating vehicle,NaN,41.9
204,Bangladesh,wheat flour,percent of population eating vehicle,NaN,76.2


In [40]:
target[target.location_name=="Bangladesh"].merge(
    subset_data['Bangladesh'][rcols + ['value_mean']],
    on = rcols,
    how = 'left'
)

,location_name,vehicle,value_description,nutrient,value_mean
0,Bangladesh,oil,percent of population eating fortified vehicle,vitamin a,42.8496
1,Bangladesh,oil,percent of population eating industrially prod...,na,88.4
2,Bangladesh,oil,percent of population eating vehicle,na,100
3,Bangladesh,wheat flour,percent of population eating fortified vehicle,folic acid,0.155875
4,Bangladesh,wheat flour,percent of population eating fortified vehicle,iron,0.155875
5,Bangladesh,wheat flour,percent of population eating fortified vehicle,vitamin a,0.155875
6,Bangladesh,wheat flour,percent of population eating fortified vehicle,zinc,0.155875
7,Bangladesh,wheat flour,percent of population eating industrially prod...,na,NaN
8,Bangladesh,wheat flour,percent of population eating vehicle,na,NaN


## Pakistan

In [41]:
pakistan = check_one_country('Pakistan')

In [42]:
pakistan.value_description.unique()

array(['percent of population eating fortified vehicle',
       'percent of population eating industrially produced vehicle',
       'percent of population eating vehicle'], dtype=object)

In [43]:
# percent of pop eating ind prod oil
pakistan[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
61,NaN,Pakistan,unknown,Balochistan,oil,percent of population eating industrially prod...,na,99,98,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
62,NaN,Pakistan,unknown,Punjab,oil,percent of population eating industrially prod...,na,99,98,100,total population,NaN,vehicle for oil for them is oil/ghee,NaN,https://www.gainhealth.org/sites/default/files...,
63,NaN,Pakistan,unknown,Sindh,oil,percent of population eating industrially prod...,na,99,98,100,total population,NaN,vehicle for oil for them is oil/ghee,NaN,https://www.gainhealth.org/sites/default/files...,
760,NaN,Pakistan,mixed/both,na,oil,percent of population eating industrially prod...,na,85,na,na,total population,2019,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/full-gfdx-datasets/,


In [44]:
#use source referring to oil as opposed to oil/ghee

pakistan.loc[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle") &
        (pakistan.source_citation=="GFDx"),"data_choice_notes"] = "Kept GFDx number (85%), discarded FACT survey number (99%), which asked about both oil and ghee"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle") &
        (pakistan.source_citation=="GFDx")])



In [45]:
# percent of pop eating oil
pakistan[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
64,NaN,Pakistan,unknown,Balochistan,oil,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
65,NaN,Pakistan,unknown,Punjab,oil,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
66,NaN,Pakistan,unknown,Sindh,oil,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,


In [46]:
## these are all the same; take one

pakistan.loc[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating vehicle") &
        (pakistan.subnational_name == "Balochistan"),"data_choice_notes"] = "Only one source"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan[(pakistan.vehicle=="oil") &
         (pakistan.value_description=="percent of population eating vehicle") &
        (pakistan.subnational_name == "Balochistan")])



In [47]:
df.loc[(df.location_name=="Pakistan") & 
  (df.vehicle=="oil") & (df.value_description=="percent of population eating fortified vehicle"),check_cols]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
67,NaN,Pakistan,unknown,Balochistan,oil,percent of population eating fortified vehicle,vitamin a,39,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,
68,NaN,Pakistan,unknown,Punjab,oil,percent of population eating fortified vehicle,vitamin a,31,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,
69,NaN,Pakistan,unknown,Sindh,oil,percent of population eating fortified vehicle,vitamin a,20,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,


In [48]:
# same source; take all

pakistan.loc[(pakistan.vehicle=="oil") & (pakistan.value_description=="percent of population eating fortified vehicle"),'data_choice_notes'] = "Only one source"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan.loc[(pakistan.vehicle=="oil") & (pakistan.value_description=="percent of population eating fortified vehicle")])



In [49]:
# percent of pop eating ind prod wheat
pakistan[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
70,NaN,Pakistan,unknown,Balochistan,wheat flour,percent of population eating industrially prod...,na,52,na,na,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
71,NaN,Pakistan,unknown,Punjab,wheat flour,percent of population eating industrially prod...,na,31,na,na,total population,NaN,vehicle for oil for them is oil/ghee,NaN,https://www.gainhealth.org/sites/default/files...,
72,NaN,Pakistan,unknown,Sindh,wheat flour,percent of population eating industrially prod...,na,63,na,na,total population,NaN,vehicle for oil for them is oil/ghee,NaN,https://www.gainhealth.org/sites/default/files...,


In [50]:
# only one source, take it

pakistan.loc[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle"),"data_choice_notes"] = "Only one source"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating industrially produced vehicle")])



In [51]:
# percent of pop eating wheat
pakistan[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
75,NaN,Pakistan,unknown,Balochistan,wheat flour,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
76,NaN,Pakistan,unknown,Punjab,wheat flour,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,
77,NaN,Pakistan,unknown,Sindh,wheat flour,percent of population eating vehicle,na,92,NaN,NaN,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,


In [52]:
#these are all the same; take one

pakistan.loc[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating vehicle") &
        (pakistan.subnational_name == "Balochistan"),"data_choice_notes"] = "Only one source"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating vehicle") &
        (pakistan.subnational_name == "Balochistan")])

pakistan[(pakistan.vehicle=="wheat flour") &
         (pakistan.value_description=="percent of population eating vehicle") &
        (pakistan.subnational_name == "Balochistan")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
75,NaN,Pakistan,unknown,Balochistan,wheat flour,percent of population eating vehicle,na,100,100,100,total population,NaN,"vehicle for oil for them is oil/ghee , Paper d...",NaN,https://www.gainhealth.org/sites/default/files...,Only one source


In [53]:
# percent of pop eating fortified wheat
# same source; take all
pakistan.loc[(pakistan.vehicle=="wheat flour") & (pakistan.value_description=="percent of population eating fortified vehicle"),"data_choice_notes"] = "Only one source"

subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan.loc[(pakistan.vehicle=="wheat flour") & (pakistan.value_description=="percent of population eating fortified vehicle")])

pakistan.loc[(pakistan.vehicle=="wheat flour") & (pakistan.value_description=="percent of population eating fortified vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
78,NaN,Pakistan,unknown,Balochistan,wheat flour,percent of population eating fortified vehicle,iron,5,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,Only one source
79,NaN,Pakistan,unknown,Punjab,wheat flour,percent of population eating fortified vehicle,iron,1,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,Only one source
80,NaN,Pakistan,unknown,Sindh,wheat flour,percent of population eating fortified vehicle,iron,10,NaN,NaN,total population,NaN,Consumption of the fortified food vehicle was ...,NaN,https://www.gainhealth.org/sites/default/files...,Only one source


In [54]:
pakistan_estimates = mult_estimates[(mult_estimates.location_name=="Pakistan") 
               & (mult_estimates.vehicle=="wheat flour")
              & (mult_estimates.nutrient.isin(['zinc','folic acid','vitamin a']))]
pakistan_estimates

,location_name,vehicle,nutrient,standard,value_description,value_mean,estimation_status
607,Pakistan,wheat flour,folic acid,Mandatory in Punjab only,percent of population eating fortified vehicle,1.946667,multiplicative
608,Pakistan,wheat flour,zinc,Unknown,percent of population eating fortified vehicle,1.946667,multiplicative


In [55]:
pakistan_estimates = pakistan_estimates.append(pd.DataFrame({
    'location_name':['Pakistan'],
    'vehicle':['wheat flour'],
    'nutrient':['vitamin a'],
    'standard':['Unknown'],
    'value_description':['percent of population eating fortified vehicle'],
    'value_mean':[0],
    'estimation_status':['lack of evidence']
}))

In [56]:
subset_data['Pakistan'] = subset_data['Pakistan'].append(pakistan_estimates)

## United Republic of Tanzania

In [57]:
tanz = check_one_country('United Republic of Tanzania')

In [58]:
tanz.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [59]:
tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
151,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,2.5,1.3,3.7,total population,2015,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
152,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,2.5,1.3,3.7,total population,2015,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
774,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,2.5,1.3,3.7,total population,2017,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
239,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
240,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
243,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
245,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,vitamin a,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [60]:
# these are all the same, take the aaron version

tanz.loc[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
     (tanz.source_year=='2011'),"data_choice_notes"] = "All sources (Aaron and GFDx) agreed."


subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
     (tanz.source_year=='2011')])

tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
     (tanz.source_year=='2011')]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
239,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
240,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
243,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
245,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating fortified vehicle,vitamin a,2.5,1.3,3.7,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [61]:
tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
150,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating industrially prod...,na,36.6,29.2,44,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
237,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating industrially prod...,na,36.6,29.2,44,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
952,NaN,United Republic of Tanzania,NaN,na,maize flour,percent of population eating industrially prod...,na,2.5,na,na,total population,2011,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [62]:
# GFDx held the same values; took this over a number from a 2011 metanalysis on the efficacy of different
# vehicles for fortification

tanz.loc[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
        "data_choice_notes"]= "GFDx held the same values; took this over a number from a 2011 metanalysis on the efficacy of different ehicles for fortification"

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
237,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating industrially prod...,na,36.6,29.2,44,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values; took this over a nu...


In [63]:
# these are the same, taking Aaron

tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
393,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating vehicle,na,93,89.7,96.4,total population,2015,Population coverage of food. Africa Academy of...,GFDx,https://fortificationdata.org/country-fortific...,
238,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating vehicle,na,93,89.7,96.4,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [64]:
tanz.loc[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
    "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="maize flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
238,NaN,United Republic of Tanzania,mixed/both,na,maize flour,percent of population eating vehicle,na,93,89.7,96.4,total population,2011,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [65]:
## oil

In [66]:
tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
290,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,53.6,46.4,60.8,total population,NaN,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
975,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,53.6,46.4,60.8,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [67]:
## these are the same, taking aaron

tanz.loc[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
     (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
    "data_choice_notes"] =  "All sources (Aaron and GFDx) agreed."


subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])


tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
     (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
975,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,53.6,46.4,60.8,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [68]:
tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating industrially produced vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
294,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating industrially prod...,na,92.6,89,96.3,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
233,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating industrially prod...,na,92.6,89,96.3,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
953,NaN,United Republic of Tanzania,NaN,na,oil,percent of population eating industrially prod...,na,54,na,na,total population,2011,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [69]:
# GFDx held the same values (93 g/day); took this over a number from a 2011 metanalysis (54 g/day) on the efficacy of different
# vehicles for fortification

tanz.loc[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
        "data_choice_notes"] = "GFDx held the same values (93%); took this over a number from a 2011 metanalysis (54%) on the efficacy of different vehicles for fortification"

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
233,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating industrially prod...,na,92.6,89,96.3,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values (93%); took this ove...


In [70]:
# these are the same, taking Aaron

tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
295,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating vehicle,na,96.2,93.2,99.2,total population,2015,Population coverage of food. Africa Academy of...,GFDx,https://fortificationdata.org/country-fortific...,
234,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating vehicle,na,96.2,93.2,99.2,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [71]:
tanz.loc[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
        "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="oil") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
234,NaN,United Republic of Tanzania,mixed/both,na,oil,percent of population eating vehicle,na,96.2,93.2,99.2,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [72]:
tanz.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [73]:
tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
624,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,folic acid,33.1,27.5,38.7,total population,NaN,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
626,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,iron,33.1,27.5,38.7,total population,NaN,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
627,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,vitamin a,33.1,27.5,38.7,total population,NaN,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
638,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,zinc,33.1,27.5,38.7,total population,NaN,Population coverage of fortified food. Africa ...,GFDx,https://fortificationdata.org/country-fortific...,
172,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,vitamin a,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
255,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,folic acid,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
256,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,iron,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
275,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,zinc,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [74]:
# all the same, taking the aaron source

tanz.loc[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
    "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating fortified vehicle") &
     (tanz.nutrient.isin(nutrients)) &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
172,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,vitamin a,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
255,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,folic acid,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
256,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,iron,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
275,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating fortified vehicle,zinc,33.1,27.5,38.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [75]:
tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
622,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,50.5,43.3,57.7,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
246,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,50.5,43.3,57.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
951,NaN,United Republic of Tanzania,NaN,na,wheat flour,percent of population eating industrially prod...,na,33,na,na,total population,2010,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [76]:
# GFDx held the same values (50.5 g/day); took this over a number from a 2011 metanalysis (33 g/day) on the efficacy of different
# vehicles for fortification

tanz.loc[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
    "data_choice_notes"] = "GFDx held the same values (51%); took this over a number from a 2011 metanalysis (33%) on the efficacy of different vehicles for fortification"

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating industrially produced vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
246,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,50.5,43.3,57.7,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values (51%); took this ove...


In [77]:
# these are the same, taking Aaron

tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
623,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating vehicle,na,51.5,44.5,58.5,total population,2015,Population coverage of food. Africa Academy of...,GFDx,https://fortificationdata.org/country-fortific...,
247,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating vehicle,na,51.5,44.5,58.5,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [78]:
tanz.loc[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770"),
    "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['United Republic of Tanzania'] = subset_data['United Republic of Tanzania'].append(tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")])

tanz[(tanz.vehicle=="wheat flour") &
     (tanz.value_description=="percent of population eating vehicle") &
    (tanz.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245770")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
247,NaN,United Republic of Tanzania,mixed/both,na,wheat flour,percent of population eating vehicle,na,51.5,44.5,58.5,total population,2010,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


## Uganda

In [79]:
uganda = check_one_country('Uganda')

uganda.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [80]:
uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating fortified vehicle") &
     (uganda.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
135,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,6.5,na,na,total population,2015,Population coverage of fortified food. Centers...,GFDx,https://fortificationdata.org/country-fortific...,
136,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,6.5,na,na,total population,2015,Population coverage of fortified food. Centers...,GFDx,https://fortificationdata.org/country-fortific...,
137,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,6.5,na,na,total population,2015,Population coverage of fortified food. Centers...,GFDx,https://fortificationdata.org/country-fortific...,
140,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,vitamin a,6.5,na,na,total population,2015,Population coverage of fortified food. Centers...,GFDx,https://fortificationdata.org/country-fortific...,
317,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
192,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
335,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
338,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,vitamin a,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [81]:
#all the same, take aaron

uganda.loc[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating fortified vehicle") &
     (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
          "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating fortified vehicle") &
     (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")])


uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating fortified vehicle") &
     (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
192,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,iron,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
317,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,folic acid,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
335,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,zinc,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
338,NaN,Uganda,mixed/both,na,maize flour,percent of population eating fortified vehicle,vitamin a,6.5,3.3,9.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [82]:
uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
133,NaN,Uganda,mixed/both,na,maize flour,percent of population eating industrially prod...,na,42.4,na,na,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
877,NaN,Uganda,mixed/both,na,maize flour,percent of population eating industrially prod...,na,42.4,32.7,52.1,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
884,NaN,Uganda,NaN,na,maize flour,percent of population eating industrially prod...,na,6.5,na,na,total population,2006,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [83]:
# GFDx held the same values (42 g/day); took this over a number from a
# 2006 datapoint (7 g/day) from a metanalysis on the efficacy of different vehicles for fortification

uganda.loc[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating industrially produced vehicle") &
    (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771"),
      "data_choice_notes"] = "GFDx held the same values (42%); took this over a number from a 2006 datapoint (7%) from a metanalysis on the efficacy of different vehicles for fortification"

subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating industrially produced vehicle") &
    (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771")])

uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating industrially produced vehicle") &
    (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
877,NaN,Uganda,mixed/both,na,maize flour,percent of population eating industrially prod...,na,42.4,32.7,52.1,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values (42%); took this ove...


In [84]:
uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
134,NaN,Uganda,mixed/both,na,maize flour,percent of population eating vehicle,na,91.8,na,na,total population,2015,Population coverage of food. Centers for Disea...,GFDx,https://fortificationdata.org/country-fortific...,
316,NaN,Uganda,mixed/both,na,maize flour,percent of population eating vehicle,na,91.8,87.7,96,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [85]:
#same values, take the aaron

uganda.loc[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771"),
      "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771")])

uganda[(uganda.vehicle=="maize flour") &
     (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_citation=="Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245771")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
316,NaN,Uganda,mixed/both,na,maize flour,percent of population eating vehicle,na,91.8,87.7,96,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [86]:
uganda.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [87]:
uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
278,NaN,Uganda,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,54.4,48.3,60.4,total population,2015,Population coverage of fortified food. Centers...,GFDx,https://fortificationdata.org/country-fortific...,
311,NaN,Uganda,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,54.4,48.3,60.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [88]:
# values the same, take the aaron

uganda.loc[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
          "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."


subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]
)

uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]


,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
311,NaN,Uganda,mixed/both,na,oil,percent of population eating fortified vehicle,vitamin a,54.4,48.3,60.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [89]:
uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
276,NaN,Uganda,mixed/both,na,oil,percent of population eating industrially prod...,na,89,na,na,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
315,NaN,Uganda,mixed/both,na,oil,percent of population eating industrially prod...,na,89,84.7,93.2,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
886,NaN,Uganda,NaN,na,oil,percent of population eating industrially prod...,na,54,na,na,total population,2011,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [90]:
# GFDx held the same values (89 g/day); took this over a number from a
# 2011 metanalysis (54 g/day) on the efficacy of different vehicles for fortification

uganda.loc[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
      "data_choice_notes"] = "GFDx held the same values (89%); took this over a number from a 2011 metanalysis (54%) on the efficacy of different vehicles for fortification"

subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")])

uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
315,NaN,Uganda,mixed/both,na,oil,percent of population eating industrially prod...,na,89,84.7,93.2,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values (89%); took this ove...


In [91]:
uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
277,NaN,Uganda,mixed/both,na,oil,percent of population eating vehicle,na,89.9,NaN,NaN,total population,2015,Population coverage of food. Centers for Disea...,GFDx,https://fortificationdata.org/country-fortific...,
314,NaN,Uganda,mixed/both,na,oil,percent of population eating vehicle,na,89.9,85.9,94,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [92]:
#values the same; taking aaron

uganda.loc[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
          "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['Uganda'] = subset_data['Uganda'].append(uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")])

uganda[(uganda.vehicle=="oil") & 
       (uganda.value_description=="percent of population eating vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
314,NaN,Uganda,mixed/both,na,oil,percent of population eating vehicle,na,89.9,85.9,94,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [93]:
uganda.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
oil         percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [94]:
## same vals, take aaron

uganda.loc[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
      "data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['Uganda'] = subset_data['Uganda'].append(
    uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]
)

uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating fortified vehicle") &
       (uganda.nutrient.isin(nutrients)) &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
342,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating fortified vehicle,folic acid,8.5,5.7,11.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
343,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating fortified vehicle,iron,8.5,5.7,11.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
344,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating fortified vehicle,vitamin a,8.5,5.7,11.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.
367,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating fortified vehicle,zinc,8.5,5.7,11.4,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


In [95]:
uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
340,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,10.6,7.6,13.6,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
600,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,10.6,na,na,total population,2015,Proportion of population consuming industriall...,GFDx,https://fortificationdata.org/country-fortific...,
882,NaN,Uganda,NaN,na,wheat flour,percent of population eating industrially prod...,na,8.5,na,na,total population,2016,Data is in table S3,NaN,https://www.mdpi.com/2072-6643/11/7/1594/htm,


In [96]:
# GFDx held the same values (10.6 g/day); took this over a number from a
# 2016 metanalysis (8.5 g/day) on the efficacy of different vehicles for fortification


uganda.loc[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),
      "data_choice_notes"] = "GFDx held the same values (11%); took this over a number from a 2016 metanalysis (9%) on the efficacy of different vehicles for fortification"

subset_data['Uganda'] = subset_data['Uganda'].append(
    uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]
)

uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating industrially produced vehicle") &
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
340,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating industrially prod...,na,10.6,7.6,13.6,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,GFDx held the same values (11%); took this ove...


In [97]:
# vals the same, take aaron

uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
341,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating vehicle,na,11.2,7.7,14.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
601,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating vehicle,na,11.2,na,na,total population,2015,Population coverage of food. Centers for Disea...,GFDx,https://fortificationdata.org/country-fortific...,


In [98]:
uganda.loc[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating vehicle") & 
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753"),"data_choice_notes"] = "All sources (Aaron and GFDx) agreed."

subset_data['Uganda'] = subset_data['Uganda'].append(
    uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating vehicle") & 
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]
)

uganda[(uganda.vehicle=="wheat flour") & 
       (uganda.value_description=="percent of population eating vehicle") & 
      (uganda.source_link=="https://doi.org/10.3945/jn.116.245753")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
341,NaN,Uganda,mixed/both,na,wheat flour,percent of population eating vehicle,na,11.2,7.7,14.7,total population,2003,NaN,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,All sources (Aaron and GFDx) agreed.


## South Africa

In [99]:
safrica = check_one_country('South Africa')

safrica.groupby(['vehicle','value_description']).mean()

location_id
vehicle     value_description                                              
maize flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN
wheat flour percent of population eating fortified vehicle              NaN
            percent of population eating industrially produ...          NaN
            percent of population eating vehicle                        NaN

In [100]:
safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating fortified vehicle")].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
1017,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,folic acid,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1018,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,iron,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1019,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,vitamin a,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1020,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,"b1, thiamine",16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1022,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,zinc,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1025,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,folic acid,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1027,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,iron,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1028,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,vitamin a,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1029,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,"b1, thiamine",77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,
1032,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,zinc,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,


In [101]:
#only one source; we'll pop-weight these

safrica.loc[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
     (safrica.nutrient.isin(nutrients)),"data_choice_notes"] = "Only one source."

subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
     (safrica.nutrient.isin(nutrients))].sort_values('source_citation'))

safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
     (safrica.nutrient.isin(nutrients))].sort_values('source_citation')

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
1017,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,folic acid,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1018,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,iron,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1019,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,vitamin a,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1022,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating fortified vehicle,zinc,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1025,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,folic acid,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1027,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,iron,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1028,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,vitamin a,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1032,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating fortified vehicle,zinc,77.4,69.8,94.9,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [102]:
#only one source (aaron); take it

safrica.loc[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle"),
           "data_choice_notes"] = "Only one source."

subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle")])

safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
1016,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating industrially prod...,na,98.7,96.5,100,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1024,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating industrially prod...,na,95.4,90.3,98.4,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [103]:
#only one source (aaron); take it

safrica.loc[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating vehicle"), "data_choice_notes"] = "Only one source."

subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating vehicle")])

safrica[(safrica.vehicle=="maize flour") &
     (safrica.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
1015,NaN,South Africa,NaN,Eastern Cape,maize flour,percent of population eating vehicle,na,98.7,96.5,100,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1023,NaN,South Africa,NaN,Gauteng,maize flour,percent of population eating vehicle,na,95.6,90.4,98.6,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [104]:
#this is somehow extracted twice, but only one source, so we'll take it

safrica.loc[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
    (safrica.nutrient.isin(nutrients)),"data_choice_notes"] = "Only one source."

subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
     (safrica.nutrient.isin(nutrients))].sort_values('source_citation').drop_duplicates())

safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating fortified vehicle") &
    (safrica.nutrient.isin(nutrients))].sort_values('source_citation').drop_duplicates()

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
978,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating fortified vehicle,folic acid,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
980,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating fortified vehicle,iron,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
982,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating fortified vehicle,vitamin a,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
995,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating fortified vehicle,zinc,16.3,10,23.7,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1002,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating fortified vehicle,folic acid,0.8,0,2.3,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1004,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating fortified vehicle,iron,0.8,0,2.3,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1006,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating fortified vehicle,vitamin a,0.8,0,2.3,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1012,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating fortified vehicle,zinc,0.8,0,2.3,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [105]:
#only one source (aaron); take it

safrica.loc[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle"),"data_choice_notes"] = "Only one source."

subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle")])

safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating industrially produced vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
977,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating industrially prod...,na,25.2,16.3,34.1,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1001,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating industrially prod...,na,4.3,1.8,7.6,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [106]:
#only one source (aaron); take it

safrica.loc[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating vehicle"),"data_choice_notes"] = "Only one source."


subset_data['South Africa'] = subset_data['South Africa'].append(safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating vehicle")])

safrica[(safrica.vehicle=="wheat flour") &
     (safrica.value_description=="percent of population eating vehicle")]

,location_id,location_name,urbanicity,subnational_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,source_year,notes,source_citation,source_link,data_choice_notes
976,NaN,South Africa,NaN,Eastern Cape,wheat flour,percent of population eating vehicle,na,25.2,16.3,34.1,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.
1000,NaN,South Africa,NaN,Gauteng,wheat flour,percent of population eating vehicle,na,4.3,1.8,7.6,total population,2015,"these are 95%, not 97.5% CIs! survey pop = hou...","Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Only one source.


In [107]:
outputs_dir = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/'

In [108]:
pct_eating = pd.read_csv(outputs_dir + 'pct_eating_regression_estimates_3_15_2021.csv')
pct_ind_prod = pd.read_csv(outputs_dir + 'pct_eating_ind_prod_regression_estimates_3_15_2021.csv')
pct_fortified = pd.read_csv(outputs_dir + 'pct_eating_fortified_regression_estimates_3_15_2021.csv')

In [109]:
pct_eating[pct_eating.location_name=="South Africa"]

,location_name,vehicle,percent_of_population_eating_vehicle
127,South Africa,oil,99.014877


In [110]:
pct_ind_prod[pct_ind_prod.location_name=="South Africa"]

,location_name,vehicle,percent_of_population_eating_industrially_produced_vehicle
20,South Africa,oil,81.716956


In [111]:
pct_fortified[pct_fortified.location_name=="South Africa"]


,location_name,vehicle,percent_of_population_eating_fortified_vehicle
20,South Africa,oil,34.013318


In [112]:
safrica_oil_estimates = pct_eating[pct_eating.location_name=="South Africa"].merge(
    pct_ind_prod[pct_ind_prod.location_name=="South Africa"], on = ['location_name','vehicle']
).merge(
    pct_fortified[pct_fortified.location_name=="South Africa"], on = ['location_name','vehicle']
)

safrica_oil_estimates = safrica_oil_estimates.melt(id_vars = ['location_name','vehicle'],
                                                   value_name = "value_mean", var_name = "value_description")

safrica_oil_estimates.value_description = safrica_oil_estimates.value_description.map({i:i.replace("_"," ") for i in safrica_oil_estimates.value_description})

safrica_oil_estimates['estimation_status'] = "regression"

In [113]:
safrica_oil_estimates

,location_name,vehicle,value_description,value_mean,estimation_status
0,South Africa,oil,percent of population eating vehicle,99.014877,regression
1,South Africa,oil,percent of population eating industrially prod...,81.716956,regression
2,South Africa,oil,percent of population eating fortified vehicle,34.013318,regression


In [114]:
safrica_oil_estimates.loc[safrica_oil_estimates.value_description=="percent of population eating fortified vehicle",'nutrient'] = 'vitamin a'
safrica_oil_estimates.loc[safrica_oil_estimates.value_description!="percent of population eating fortified vehicle",'nutrient'] = 'na'

In [115]:
safrica_oil_estimates

,location_name,vehicle,value_description,value_mean,estimation_status,nutrient
0,South Africa,oil,percent of population eating vehicle,99.014877,regression,na
1,South Africa,oil,percent of population eating industrially prod...,81.716956,regression,na
2,South Africa,oil,percent of population eating fortified vehicle,34.013318,regression,vitamin a


In [116]:
subset_data['South Africa'] = subset_data['South Africa'].append(safrica_oil_estimates)

## pop-weight subnationals

In [117]:
scols = rcols # + ['source_link','source_citation','estimation_status','sub_population']

In [118]:
df[df.location_name.isin(location_names)].subnational_name.unique()

array(['Barisal', 'Chittagong', 'Dhaka', 'Khulna', 'Rajshahi', 'Rangpur',
       'Sylhet', 'na', 'Balochistan', 'Punjab', 'Sindh', 'Eastern Cape',
       'Gauteng'], dtype=object)

In [119]:
loc_metadata = get_locs(location_set_id=35, gbd_round_id=6, decomp_step="step4")

#combine subnational estimates by population-weighting

subnats = loc_metadata[loc_metadata.location_name.isin(['Balochistan', 'Punjab', 'Sindh', 'Eastern Cape', 'Gauteng']) & 
                      (loc_metadata.parent_id.isin([165, 196]))][['location_id','location_name','parent_id']]

subnats = subnats.rename(columns = {
    'location_id':'subnational_id',
    'location_name':'subnational_name'
})

subnat_pop = get_population(age_group_id=22, 
                     location_id=list(subnats.subnational_id),
                     year_id=2017,
                     sex_id=3,
                     gbd_round_id=6, 
                     decomp_step='step5')

subnats = subnats.merge(subnat_pop, left_on = 'subnational_id', right_on = 'location_id')[['subnational_id','subnational_name','population','parent_id']]

subnats['pop_denom'] = subnats.groupby('parent_id').transform('sum').population

subnats['subnat_pop_weight'] = subnats.population / subnats.pop_denom

In [120]:
subnats

,subnational_id,subnational_name,population,parent_id,pop_denom,subnat_pop_weight
0,53616,Balochistan,1.245438e+07,165,1.70631e+08,0.0729903
1,53620,Punjab,1.102238e+08,165,1.70631e+08,0.645979
2,53621,Sindh,4.795248e+07,165,1.70631e+08,0.281031
3,482,Eastern Cape,6.493595e+06,196,2.05087e+07,0.316626
4,484,Gauteng,1.401513e+07,196,2.05087e+07,0.683374


In [121]:
tmp = subset_data['Pakistan']

In [122]:
tmp.loc[tmp.source_citation.isna(),'source_citation'] = 'na'
tmp.loc[tmp.source_link.isna(),'source_link'] = 'na'
tmp.loc[tmp.estimation_status.isna(),'estimation_status'] = 'na'

In [123]:
tmp.loc[tmp.value_025_percentile=='na','value_025_percentile'] = np.nan
tmp.loc[tmp.value_975_percentile=='na','value_975_percentile'] = np.nan

In [124]:
tmp.loc[tmp.estimation_status.isna(),'estimation_status'] = 'na'

In [125]:
tmp['is_dupl'] = tmp.duplicated(subset = rcols, keep = False)

In [126]:
tmp = tmp.merge(subnats[['subnational_name','subnat_pop_weight']], how = 'left')

In [127]:
tmp.subnat_pop_weight = tmp.subnat_pop_weight.astype(float)
tmp.value_mean = tmp.value_mean.astype(float)
tmp.value_025_percentile = tmp.value_025_percentile.astype(float)
tmp.value_975_percentile = tmp.value_975_percentile.astype(float)

In [128]:
tmp.loc[tmp.is_dupl,'value_mean'] = tmp.loc[tmp.is_dupl].value_mean * tmp.loc[tmp.is_dupl].subnat_pop_weight
tmp.loc[tmp.is_dupl,'value_025_percentile'] = tmp.value_025_percentile * tmp.subnat_pop_weight
tmp.loc[tmp.is_dupl,'value_975_percentile'] = tmp.value_975_percentile * tmp.subnat_pop_weight

In [129]:
tmp.loc[~tmp.is_dupl,'subnational_name'] = 'na'

In [130]:
tmp.value_mean = tmp.groupby(rcols).transform('sum').value_mean
tmp.value_025_percentile = tmp.groupby(rcols).transform('sum').value_025_percentile
tmp.value_975_percentile = tmp.groupby(rcols).transform('sum').value_975_percentile

In [131]:
tmp = tmp[['location_name','vehicle','value_description','nutrient','estimation_status','source_link','source_year','sub_population','value_mean','value_025_percentile','value_975_percentile','data_choice_notes']].drop_duplicates()

In [132]:
subset_data['Pakistan'] = tmp

In [133]:
## south africa

In [134]:
tmp = subset_data['South Africa']
tmp.loc[tmp.source_citation.isna(),'source_citation'] = 'na'
tmp.loc[tmp.source_citation.isna(),'source_link'] = 'na'

In [135]:
tmp['is_dupl'] = tmp.duplicated(subset = rcols, keep = False)

In [136]:
tmp = tmp.merge(subnats[['subnational_name','subnat_pop_weight']], how = 'left')

In [137]:
tmp.subnat_pop_weight = tmp.subnat_pop_weight.astype(float)
tmp.value_mean = tmp.value_mean.astype(float)
tmp.value_025_percentile = tmp.value_025_percentile.astype(float)
tmp.value_975_percentile = tmp.value_975_percentile.astype(float)

In [138]:
tmp.loc[tmp.is_dupl,'value_mean'] = tmp.loc[tmp.is_dupl].value_mean * tmp.loc[tmp.is_dupl].subnat_pop_weight
tmp.loc[tmp.is_dupl,'value_025_percentile'] = tmp.value_025_percentile * tmp.subnat_pop_weight
tmp.loc[tmp.is_dupl,'value_975_percentile'] = tmp.value_975_percentile * tmp.subnat_pop_weight

In [139]:
tmp.loc[~tmp.is_dupl,'subnational_name'] = 'na'

In [140]:
tmp.value_mean = tmp.groupby(rcols).transform('sum').value_mean
tmp.value_025_percentile = tmp.groupby(rcols).transform('sum').value_025_percentile
tmp.value_975_percentile = tmp.groupby(rcols).transform('sum').value_975_percentile

In [141]:
tmp = tmp[['location_name','vehicle','value_description','nutrient','estimation_status','source_link','source_year','sub_population','value_mean','value_025_percentile','value_975_percentile','data_choice_notes']].drop_duplicates()

In [142]:
subset_data['South Africa'] = tmp

## Check for missingness

In [143]:
all_data = pd.concat(list(subset_data.values()))

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [144]:
all_data.data_choice_notes.unique()

array(['Keeping Aaron number (88%) and descarding Raghavan number (87%), as they are very similar, from the same year, and the Aaron number has uncertainty',
       'Keeping Aaron (survey data) number from 2015 (100%), discarding Fiedler number (51-90%) from 2005, and discarding Levyraz number from 2011 (99.8%)',
       'Keeping Levyraz numbers (42% and 76%), as they are u5- and WRA- specfic and more recent (2011). Discarding 2005 Fiedler all-populaion numbers',
       nan,
       'Kept GFDx number (85%), discarded FACT survey number (99%), which asked about both oil and ghee',
       'Only one source', 'All sources (Aaron and GFDx) agreed.',
       'GFDx held the same values; took this over a number from a 2011 metanalysis on the efficacy of different ehicles for fortification',
       'GFDx held the same values (93%); took this over a number from a 2011 metanalysis (54%) on the efficacy of different vehicles for fortification',
       'GFDx held the same values (51%); took this over 

In [145]:
all_data.loc[~(all_data.nutrient.isin(['vitamin a','iron','zinc','folic acid'])),'nutrient'] = 'na'

In [146]:
all_data[rcols + ['value_mean','value_025_percentile','value_975_percentile','sub_population']]

,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population
35,Bangladesh,oil,percent of population eating industrially prod...,na,88.4,84.5,92.3,total population
34,Bangladesh,oil,percent of population eating vehicle,na,100,na,na,total population
203,Bangladesh,wheat flour,percent of population eating vehicle,na,41.9,na,na,women of reproductive age
204,Bangladesh,wheat flour,percent of population eating vehicle,na,76.2,na,na,under-5
369,Bangladesh,oil,percent of population eating fortified vehicle,vitamin a,42.8496,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
20,South Africa,wheat flour,percent of population eating industrially prod...,na,10.9175,6.39108,15.9906,total population
22,South Africa,wheat flour,percent of population eating vehicle,na,10.9175,6.39108,15.9906,total population
24,South Africa,oil,percent of population eating vehicle,na,99.0149,0,0,NaN
25,South Africa,oil,percent of population eating industrially prod...,na,81.717,0,0,NaN


In [147]:
check = target.merge(all_data[rcols + ['value_mean']], on = rcols, how = 'left')

In [148]:
check[check.value_mean.isna()]

,location_name,vehicle,value_description,nutrient,value_mean


In [149]:
assert(len(check[check.value_mean.isna()])==0), "there are target loc/vehcile/val/nutrient combos youre missing"

In [150]:
all_data.columns

Index(['data_choice_notes', 'estimation_status', 'location_id',
       'location_name', 'notes', 'nutrient', 'source_citation', 'source_link',
       'source_year', 'standard', 'sub_population', 'subnational_name',
       'urbanicity', 'value_025_percentile', 'value_975_percentile',
       'value_description', 'value_mean', 'vehicle'],
      dtype='object')

In [151]:
output = all_data[rcols + ['value_mean','value_025_percentile', 'value_975_percentile','sub_population','estimation_status','source_citation','source_link','data_choice_notes']].sort_values(rcols).set_index(rcols)

In [152]:
## TODO

In [153]:
output[(output.estimation_status.isna()) & ((output.source_citation.isna()) | (output.source_link.isna()))]

value_mean  \
location_name vehicle     value_description                                  nutrient                
South Africa  maize flour percent of population eating fortified vehicle     folic acid    58.0541   
                                                                             iron          58.0541   
                                                                             vitamin a     58.0541   
                                                                             zinc          58.0541   
                          percent of population eating industrially produ... na            96.4449   
                          percent of population eating vehicle               na            96.5815   
              wheat flour percent of population eating fortified vehicle     folic acid     5.7077   
                                                                             iron           5.7077   
                                                                             vitamin a      5.7077   
                                                                             zinc           5.7077   
                          percent of population eating industrially produ... na            10.9175   
                          percent of population eating vehicle               na            10.9175   

                                                                                        value_025_percentile  \
location_name vehicle     value_description                                  nutrient                          
South Africa  maize flour percent of population eating fortified vehicle     folic acid              50.8658   
                                                                             iron                    50.8658   
                                                                             vitamin a               50.8658   
                                                                             zinc                    50.8658   
                          percent of population eating industrially produ... na                      92.2631   
                          percent of population eating vehicle               na                      92.3314   
              wheat flour percent of population eating fortified vehicle     folic acid              3.16626   
                                                                             iron                    3.16626   
                                                                             vitamin a               3.16626   
                                                                             zinc                    3.16626   
                          percent of population eating industrially produ... na                      6.39108   
                          percent of population eating vehicle               na                      6.39108   

                                                                                        value_975_percentile  \
location_name vehicle     value_description                                  nutrient                          
South Africa  maize flour percent of population eating fortified vehicle     folic acid              72.3562   
                                                                             iron                    72.3562   
                                                                             vitamin a               72.3562   
                                                                             zinc                    72.3562   
                          percent of population eating industrially produ... na                      98.9066   
                          percent of population eating vehicle               na                      99.0433   
              wheat flour percent of population eating fortified vehicle     folic acid               9.0758   
                                                                             iron 

In [154]:
df[df.source_link=="https://doi.org/10.3945/jn.116.245753"].source_citation.unique()[0]

'Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245753'

In [155]:
output.loc[(output.estimation_status.isna()) & ((output.source_citation.isna()) & (output.source_link=="https://doi.org/10.3945/jn.116.245753")),'source_citation'] = 'Grant J Aaron, Valerie M Friesen, Svenja Jungjohann, Greg S Garrett, Lynnette M Neufeld, Mark Myatt, Coverage of Large-Scale Food Fortification of Edible Oil, Wheat Flour, and Maize Flour Varies Greatly by Vehicle and Country but Is Consistently Lower among the Most Vulnerable: Results from Coverage Surveys in 8 Countries, The Journal of Nutrition, Volume 147, Issue 5, May 2017, Pages 984S–994S, https://doi.org/10.3945/jn.116.245753'

In [156]:
output = output.reset_index()

In [157]:
##impute all missing CIs

In [158]:
# clean value_mean
output.loc[output.value_mean=='na','value_mean'] = np.nan
output.value_mean = output.value_mean.astype(float)

# clean 2.5th %ile
output.loc[output.value_025_percentile=='na','value_025_percentile'] = np.nan
output.value_025_percentile = output.value_025_percentile.astype(float)

# clean 97.5th %ile
output.loc[output.value_975_percentile=='na','value_975_percentile'] = np.nan
output.value_975_percentile = output.value_975_percentile.astype(float)


In [159]:
output['scale_over_mean'] = (output.value_975_percentile - output.value_025_percentile) / output.value_mean

In [160]:
r = output[['vehicle','scale_over_mean']].groupby('vehicle').mean().dropna().rename(columns={'scale_over_mean':'r'})

In [161]:
# add uncertainty
output = output.merge(r, on = 'vehicle', how = 'outer')
output['lower'] = np.clip(output.value_mean - (output.r * output.value_mean)/2, 0, 100)
output['upper'] = np.clip(output.value_mean + (output.r * output.value_mean)/2, 0, 100)

In [162]:
output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'CI_source'] = "modeling"
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'CI_source'] = "modeling"

In [163]:
output.loc[output.CI_source.isna(),'CI_source'] = 'extraction'

In [164]:
output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna()),'value_025_percentile'] = output.loc[(output.value_mean < output.value_025_percentile) | (output.value_025_percentile.isna())].lower
output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna()),'value_975_percentile'] = output.loc[(output.value_mean > output.value_975_percentile) | (output.value_975_percentile.isna())].upper

In [165]:
output = output.drop(columns=['r','lower','upper','scale_over_mean']).set_index(rcols)

In [166]:
output.loc[output.estimation_status.isna(),'estimation_status'] = 'na'

/ihme/homes/beatrixh/.local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2848: PerformanceWarning: indexing past lexsort depth may impact performance.
  raw_cell, store_history, silent, shell_futures)


In [167]:
save_path_tmp = '/ihme/homes/beatrixh/repos/scratch/pop_coverage_data_with_data_choice_notes_3_17_2021.csv'
output.to_csv(save_path_tmp)

In [168]:
## add loc_ids

In [169]:
output = loc_metadata[['location_id','location_name']].merge(output.reset_index(), on = 'location_name', how = 'right')

In [170]:
save_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/population_coverage_data_tier2_locs_3_17_2021.csv'
output.to_csv(save_path, index = False)

In [171]:
output.head()

,location_id,location_name,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile,sub_population,estimation_status,source_citation,source_link,data_choice_notes,CI_source
0,161,Bangladesh,oil,percent of population eating fortified vehicle,vitamin a,42.849572,41.356218,44.342926,NaN,multiplicative,NaN,NaN,NaN,modeling
1,161,Bangladesh,oil,percent of population eating industrially prod...,na,88.400000,84.500000,92.300000,total population,na,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Keeping Aaron number (88%) and descarding Ragh...,extraction
2,161,Bangladesh,oil,percent of population eating vehicle,na,100.000000,96.514892,100.000000,total population,na,"Grant J Aaron, Valerie M Friesen, Svenja Jungj...",https://doi.org/10.3945/jn.116.245753,Keeping Aaron (survey data) number from 2015 (...,modeling
3,161,Bangladesh,wheat flour,percent of population eating fortified vehicle,folic acid,0.155875,0.116283,0.195467,NaN,multiplicative,NaN,NaN,NaN,modeling
4,161,Bangladesh,wheat flour,percent of population eating fortified vehicle,iron,0.155875,0.116283,0.195467,NaN,multiplicative,NaN,NaN,NaN,modeling


In [173]:
formatted_cols = ['location_id','location_name','sub_population','vehicle','value_description','nutrient','value_mean','value_025_percentile','value_975_percentile']

save_path = '/ihme/homes/beatrixh/vivarium_research_lsff/data_prep/outputs/population_coverage_data_tier2_lo.csv'
output[formatted_cols]

,location_id,location_name,sub_population,vehicle,value_description,nutrient,value_mean,value_025_percentile,value_975_percentile
0,161,Bangladesh,NaN,oil,percent of population eating fortified vehicle,vitamin a,42.849572,41.356218,44.342926
1,161,Bangladesh,total population,oil,percent of population eating industrially prod...,na,88.400000,84.500000,92.300000
2,161,Bangladesh,total population,oil,percent of population eating vehicle,na,100.000000,96.514892,100.000000
3,161,Bangladesh,NaN,wheat flour,percent of population eating fortified vehicle,folic acid,0.155875,0.116283,0.195467
4,161,Bangladesh,NaN,wheat flour,percent of population eating fortified vehicle,iron,0.155875,0.116283,0.195467
...,...,...,...,...,...,...,...,...,...
59,196,South Africa,total population,maize flour,percent of population eating fortified vehicle,iron,58.054148,50.865762,72.356225
60,196,South Africa,total population,maize flour,percent of population eating fortified vehicle,vitamin a,58.054148,50.865762,72.356225
61,196,South Africa,total population,maize flour,percent of population eating fortified vehicle,zinc,58.054148,50.865762,72.356225
62,196,South Africa,total population,maize flour,percent of population eating industrially prod...,na,96.444866,92.263081,98.906602
